In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Imports

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import confusion_matrix, classification_report

# Load the data

In [ ]:
import csv
emotion_data = pd.read_csv("/kaggle/input/eeg-brainwave-dataset-feeling-emotions/emotions.csv")

In [ ]:
emotion_data

In [ ]:
sample = emotion_data.loc[0, 'fft_0_b':'fft_749_b']

plt.figure(figsize=(16, 10))
plt.plot(range(len(sample)), sample)
plt.title("Features fft_0_b through fft_749_b")
plt.show()

In [ ]:
maps = dict({"NEGATIVE":2,"POSITIVE":1,"NEUTRAL":0})


In [ ]:
y = pd.get_dummies(y)

# Split data into training and testing

In [ ]:
from sklearn.preprocessing import OneHotEncoder
y = pd.get_dummies(emotion_data['label'].copy().apply(lambda x: maps[x]))
X = emotion_data.drop('label', axis=1).copy()
# enc = OneHotEncoder(handle_unknown='ignore')
# y = tf.one_hot(y,3)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [ ]:
import pandas as pd
from sklearn import preprocessing

x = X_train #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

# SVM

In [ ]:
# Doesn't work when one hot encoding
import sklearn
from sklearn.svm import SVC
clf = SVC()
clf.fit(df,y_train)

In [ ]:
clf.score(X_test,y_test)

In [ ]:
y_pred = clf.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))

#
       #Happy Sad Neutral
#Happy  200    10   230
#Sad
#Neutral

# Neural Network

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import keras


model = models.Sequential()
model.add(tf.keras.Input(shape=(X_train.shape[1],)))
model.add(tf.keras.layers.Dense(2000, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation="softmax"))

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print(model.summary())

In [ ]:
model.fit(df,y_train,epochs=30,batch_size=32)

In [ ]:
import pandas as pd
from sklearn import preprocessing

x = X_test #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

model.evaluate(df,y_test)

# CNN

In [ ]:
print(X_train.shape)
temp_X = tf.expand_dims(df,2)
print(temp_X.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt


model = models.Sequential()
model.add(layers.Conv1D(16, 10, activation='relu',input_shape=(2548,1)))
model.add(layers.Conv1D(16, 3, activation='relu'))
model.add(layers.MaxPool1D(2))
model.add(layers.Flatten())

model.add(layers.Dense(100,activation="relu"))
model.add(layers.Dense(3,activation="softmax"))

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

In [ ]:
model.fit(temp_X,y_train,epochs=32,batch_size=32)